In [3]:
speeduplog = {}
def add_comparison(operation: str, base: float, new: float):
    speeduplog[operation] = str(round(base / new, 2)) + 'x'
    print(pd.DataFrame.from_dict(speeduplog, orient='index', columns=['speedup']))

In [1]:
!pip install Cython
!pip install scipy

You should consider upgrading via the 'c:\users\nielsota\code\statespace\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\nielsota\code\statespace\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

In [4]:
%load_ext cython

In [5]:
%reload_ext cython

In [6]:
import Cython
import cython
import scipy

In [7]:
scipy.__version__

'1.8.0'

In [8]:
Cython.__version__

'0.29.28'

In [9]:
def fib(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        a, b = b, a + b
    return a

In [10]:
%timeit fib(1e8)

3.03 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
%%cython
def cfib(int n):
    """Print the Fibonacci series up to n."""
    cdef int a = 1
    cdef int b = 1
    while b < n:
        a, b = b, a + b
    return a

In [12]:
%timeit cfib(1e8)

125 ns ± 6.88 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


# $ABC^T$

In [ ]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cmm(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    # M3 is where output is stored
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &K, &beta,
                                        &M3[0,0], &M,)
    
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cmm_transpose(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)
    
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cmm3(double[::1, :] A, double[::1, :] B, double[::1, :] C, double[::1, :] AB, double[::1, :] ABC):
    """
    compute matrix product of A, B and C' -> ABC' 
    """
    
    cmm(A, B, AB)   # result stored in out = 1*AB + 0*out = AB
    cmm(AB, C, ABC) # result stored in out = 1*ABC^T + 0*(AB)
    
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
def loop_cmm3(int T, double[::1, :] A, double[::1, :] B, double[::1, :] C, double[::1, :] AB, double[::1, :] ABC):
    
    cdef size_t i
        
    for i in range(T):
        #C = np.empty((10, 10), dtype=np.float64, order='F')
        cmm3(A, B, C, AB, ABC)
    
    return C

In [ ]:
A = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
B = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
C = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
out = np.empty((10, 10), dtype=np.float64, order='F')

In [ ]:
T = np.eye(10, dtype=np.float64, order='F') * 2
P = np.eye(10, dtype=np.float64, order='F') * 2
T_t = np.eye(10, dtype=np.float64, order='F')
TP = np.empty((10, 10), dtype=np.float64, order='F')
TPT_t = np.empty((10, 10), dtype=np.float64, order='F')

In [ ]:
%timeit loop_cmm3(10000, T, P, T_t, TP, TPT_t)

In [ ]:
%timeit loop_mm3(m1_test, m2_test, m3_test, 10000)

In [ ]:
add_comparison('MM3 multiplication (ABC\')', 42.4, 8.02)

# Start of custom Kalman Implementation

## required 

- $v= y-Za_t$: so subtraction and dot product

- $F_t = ZP_tZ' + H$: so vector matrix vector and addition

- $M = P_tZ'F_t$: matrix * vector * number

- $K_t = TM_t$: matrix vector
- $ a_{t+1} = Ta_t + Kv $: matrix vector + vector number
- $ P_{t+1} = TP_tT' + RQR' - KF_tK'$: matrix3, scaled outer product

### dot product

In [55]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cddot(int N, double[::1] x, int incX, double[::1] y, int incY) nogil:
    """
    
    custom dot product
        
        arguments
            n: length of the vectors x and y
        
        returns: type double
        
    """
    return ddot(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double c_v(double[::1] Z, double[::1, :] a, double[::1, :] y, double[::1, :] v, int incX, double[::1] y, int incY) nogil:
    """
    
    custom dot product
        
        arguments
            n: length of the vectors x and y
        
        returns: type double
        
    """
    return ddot(&N, &x[0], &incX, &y[0], &incY)

## matrix vector

In [68]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemv, ddot
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void cdgemv(double[::1, :] A, double[::1] x, double[::1] y, int incX=1, int incY=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double c_xAx(double[::1, :] A, double[::1] x, double[::1] Ax, double[::1] xAx, int incX=1, int inc_Ax=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector x'Ax
    
        output: xAx
        
        returns: type double (can do arithmetic in C)
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &Ax[0], &inc_Ax)
    return ddot(&M, &x[0], &incX, &Ax[0], &inc_Ax)


### matrix multiplication

In [ ]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cmm(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)
    
    return 0


### matrix multiplication (transpose)

In [27]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cmm_transpose(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    """
    ABC'
    """
    
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)
    
    return 0

# tests

### dot product test

In [59]:
x = np.ones(1)
y = np.ones(1)

In [60]:
x.shape

(1,)

In [61]:
cddot(2, x , 1, y, 1)

1.0

In [16]:
%timeit np.dot(x,y)

1.7 µs ± 93.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### matrix matrix test

In [26]:
# result should be all 5's in left hand side, zeros on right hand side

A = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
B = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
AB = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')

#output stored in AB
cmm(A, B, AB)
C

array([[5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.],
       [5., 5., 5., 5., 5., 0., 0., 0., 0., 0.]])

### matrix matrix transpose test

In [30]:
# result should be all 5's 

A = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
B = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
AB_transpose = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')

#output stored in AB
cmm_transpose(A, B, AB_transpose)
C

array([[5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]])

### matrix vector test

In [39]:
A = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
x = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
Ax = np.empty((10,), dtype=np.float64, order='F')

In [42]:
cddot(A, x, Ax)

In [43]:
# should be all 5's
Ax

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

### $x'Ax$ test

In [71]:
P = np.full((10, 10), [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
Z = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=np.float64, order='F')
PZ = np.empty((10,), dtype=np.float64, order='F')
ZPZ = np.empty((10,), dtype=np.float64, order='F')

In [72]:
# should be 25
c_xAx(P, Z, PZ, ZPZ)

25.0

### $v = y - Za$ test

In [75]:
T = 100

# Z: s x p
Z = np.array([1, 1], dtype=np.float64, order='F')

# a: p x T
a = np.ones((2,T), dtype=np.float64, order='F')

# y: s x T
y = np.ones((1,T), dtype=np.float64, order='F')

v = np.empty((1, T), dtype=np.float64, order='F')